In [3]:
import numpy
from matplotlib import pyplot
from pandas import read_csv
import math
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
%matplotlib inline
dataframe1 = read_csv('DailyData1.csv', engine='python')
dataframe = dataframe1.iloc[:,2:]
dataset = dataframe.values
# 将整型变为float
dataset = dataset.astype('float64')
dataframe1.head()

,Date,labels,cmeclose,cmcclose,google,ravenpack,XAUprice,cmcvolume,market,AveBlockSize,...,CostPerTran,FeePerTran,MinRevenue,NetDifficulty,TolHashRate,TolTranFee,ConTranPerDay,EstTranValue,OutValue,UniAddressUsed
0,2017-12-19,-1,17545,17776.7,52,0.24737,1261.72,16894499840,3.162220e+11,1.060159,...,116.815051,32.003749,44228748.17,1873110000000,13035760.95,12117323.34,378622,3.906879e+09,1984897.730,887698
1,2017-12-20,-1,16490,16624.6,69,0.23034,1265.62,22149699584,2.775528e+11,1.072800,...,119.379078,41.644261,44203326.97,1873110000000,12663310.64,15419911.88,370277,5.449321e+09,2227875.178,853864
2,2017-12-21,-1,15555,15802.9,54,0.17781,1266.76,16516599808,2.781525e+11,1.038692,...,135.173009,54.790393,46165232.30,1873110000000,12290860.32,18712398.65,335482,3.765982e+09,1791086.429,840740
3,2017-12-22,-1,15555,13831.8,100,0.13745,1275.01,22197999616,2.602869e+11,1.062319,...,125.710492,54.637979,48973790.42,1873110000000,14432449.62,21285645.39,380648,4.135072e+09,1901698.950,890732
4,2017-12-23,1,15555,14699.2,63,0.08594,1275.01,13086000128,2.332497e+11,1.060038,...,135.118563,51.402623,41645027.49,1873110000000,12942648.37,15842853.90,308211,2.381108e+09,1781229.264,729638


In [4]:
dataset.shape

(834, 23)

In [5]:
labels = dataframe1.iloc[:,1:2].values
scaler1 = OneHotEncoder()
labels = scaler1.fit_transform(labels)

labels=labels.toarray()

In [6]:
def create_dataset(dataset,dataset1,look_back):
    dataX, dataY = [], []
    for i in range(0,len(dataset)-look_back,1):
        a1 = dataset[i:(i+look_back),:]
        a2 = dataset1[i:(i+look_back),:]
        a = numpy.hstack((a1,a2))
        dataX.append(a)
        b = dataset1[(i+look_back):(i+look_back+1),1]
        dataY.append(b)
    return numpy.array(dataX), numpy.array(dataY)

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [8]:
look_back = 20
look_lag = 1 #用前20个数据预测当前1个数据
train_size = 626
test_size = len(dataset) - train_size
train, test = dataset[:train_size,:], dataset[train_size-look_back:len(dataset),:]
train1, test1 = labels[:train_size,:], labels[train_size-look_back:len(dataset),:]
trainX, trainY = create_dataset(train,train1,look_back)
testX, testY = create_dataset(test, test1 ,look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], look_back,25))
testX = numpy.reshape(testX, (testX.shape[0],look_back,25))

features_set = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1]*25))
scaler1 = StandardScaler().fit(features_set) #MinMaxScaler(feature_range=(-1, 1))
features_set = scaler1.fit_transform(features_set)
#pca=PCA(n_components=500).fit(features_set)
#features_set = pca.transform(features_set)
test_features = numpy.reshape(testX, (testX.shape[0], testX.shape[1]*25))
test_features = scaler1.fit_transform(test_features)
#test_features = pca.transform(test_features)

def result(testY,testPredict):
    from sklearn.metrics import accuracy_score,classification_report
    acc = accuracy_score(testY,testPredict)
    report = classification_report(testY,testPredict)
    print("acc:",acc)
    print(report)

trainY = trainY.reshape(trainY.shape[0])
testY = testY.reshape(testY.shape[0])
trainY

array([0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 0., 1.

In [24]:
from sklearn.linear_model import LogisticRegression as LR
model1 = LR() #C=100,max_iter=5000
model1.fit(features_set, trainY)
testPredict1 = model1.predict(test_features)  

/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [25]:
result(testY,testPredict1)

acc: 0.5432692307692307
              precision    recall  f1-score   support

         0.0       0.60      0.52      0.56       115
         1.0       0.49      0.57      0.53        93

    accuracy                           0.54       208
   macro avg       0.55      0.55      0.54       208
weighted avg       0.55      0.54      0.54       208



In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model2 = LinearDiscriminantAnalysis() 
model2.fit(features_set, trainY)
testPredict2 = model2.predict(test_features)  
result(testY,testPredict2)

acc: 0.5192307692307693
              precision    recall  f1-score   support

         0.0       0.57      0.52      0.55       115
         1.0       0.47      0.52      0.49        93

    accuracy                           0.52       208
   macro avg       0.52      0.52      0.52       208
weighted avg       0.52      0.52      0.52       208



In [27]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model3 = QuadraticDiscriminantAnalysis() 
model3.fit(features_set, trainY)
testPredict3 = model3.predict(test_features)  
result(testY,testPredict3)

acc: 0.5528846153846154
              precision    recall  f1-score   support

         0.0       0.60      0.57      0.58       115
         1.0       0.50      0.54      0.52        93

    accuracy                           0.55       208
   macro avg       0.55      0.55      0.55       208
weighted avg       0.56      0.55      0.55       208



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [13]:
!pip install xgboost

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [28]:
import xgboost as xgb
model4 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5, learning_rate = 0.003,
                max_depth = 20, alpha = 20, n_estimators = 400)
'''
objective ='reg:logistic', colsample_bytree = 0.5, learning_rate = 0.003,
                max_depth = 20, alpha = 20, n_estimators = 400
'''
model4.fit(features_set, trainY)
testPredict4 = model4.predict(test_features)  
result(testY,testPredict4)

acc: 0.5528846153846154
              precision    recall  f1-score   support

         0.0       0.61      0.55      0.58       115
         1.0       0.50      0.56      0.53        93

    accuracy                           0.55       208
   macro avg       0.55      0.55      0.55       208
weighted avg       0.56      0.55      0.55       208



In [29]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(features_set, trainY)
testPredict5 = model5.predict(test_features)  
result(testY,testPredict5)

acc: 0.5480769230769231
              precision    recall  f1-score   support

         0.0       0.61      0.50      0.55       115
         1.0       0.50      0.60      0.54        93

    accuracy                           0.55       208
   macro avg       0.55      0.55      0.55       208
weighted avg       0.56      0.55      0.55       208



In [30]:
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier()#n_estimators=400,criterion="entropy"
regr.fit(features_set, trainY)

RandomForestClassifier()

In [31]:
testPredict6 = regr.predict(test_features)  
result(testY,testPredict6)

acc: 0.5288461538461539
              precision    recall  f1-score   support

         0.0       0.59      0.47      0.52       115
         1.0       0.48      0.60      0.53        93

    accuracy                           0.53       208
   macro avg       0.54      0.54      0.53       208
weighted avg       0.54      0.53      0.53       208



In [18]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 750 kB/s eta 0:00:01
     |████████████████████████████████| 42 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 2.9 MB/s eta 0:00:01


In [19]:
import numpy
import keras
from matplotlib import pyplot
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Conv1D,Flatten
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from keras.models import load_model
%matplotlib inline
from keras import optimizers,initializers
model = Sequential() 

model.add(LSTM(units=96, return_sequences=True, 
               input_shape=(trainX.shape[1],trainX.shape[2])))#
model.add(Dropout(0.3))

model.add(LSTM(units=48, return_sequences=True,activation='relu'))
model.add(Dropout(0.1))
model.add(LSTM(units=48,return_sequences=False,activation='relu'))
model.add(Dropout(0.1))

'''
model.add(Conv1D(kernel_size=20, filters=3,activation='relu'))
model.add(Flatten())
'''

#model.add(Dense(units = 16,activation='relu'))
model.add(Dense(units = 1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',#binary_crossentropy
                  optimizer='adam',#RMSprop optimizers.Adam(lr=0.00001)
                  metrics=['accuracy'])  #mean_squared_error metrics=['accuracy']
history = model.fit(trainX,trainY,epochs=500, batch_size=64, shuffle=True)

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [20]:
testPredict = model.predict_classes(testX)
result(testY,testPredict)

acc: 0.5144230769230769
              precision    recall  f1-score   support

         0.0       0.59      0.42      0.49       115
         1.0       0.47      0.63      0.54        93

    accuracy                           0.51       208
   macro avg       0.53      0.53      0.51       208
weighted avg       0.53      0.51      0.51       208

